In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import math
from sklearn import linear_model

In [ ]:
DATA_CSV = 'data/ME_clean_nh.csv'

In [ ]:
raw_points = pd.read_csv(DATA_CSV, index_col='py')

In [ ]:
REQ_Y = 5
prev_py = None
prev_carb = None
data_points = []
total_diff_y = 0.0
for py, row in raw_points.iterrows():
    if (prev_py != None) and (prev_py // 10000 == py // 10000) and (py - prev_py == REQ_Y):
        data_points.append([[row['lon'], row['lat'], prev_carb], row['carb']])
        total_diff_y += py - prev_py
    prev_py = py
    prev_carb = row['carb']
total_diff_y /= len(data_points)

In [ ]:
N_GROUPS = 10
random.shuffle(data_points)
groups = []
prev_cutoff = 0
for i in np.arange(N_GROUPS):
    next_cutoff = (i + 1) * len(data_points) / N_GROUPS
    groups.append(data_points[prev_cutoff:next_cutoff])
    prev_cutoff = next_cutoff

In [ ]:
min_mse = float("inf")
for i in np.arange(N_GROUPS):
    training = []
    test = []
    for j, group in enumerate(groups):
        if j==i:
            test = group
        else:
            training.extend(group)
    training = np.array(training).T.tolist()
    clf = linear_model.LinearRegression()
    clf.fit(training[0], training[1])
    coef = clf.coef_
    cons = clf.intercept_
    sq_error = 0.0
    for elem in test:
        predicted = sum(coef * elem[0]) + cons
        actual = elem[1]
        sq_error += (actual - predicted) ** 2
    mse = math.sqrt(sq_error/len(test))
    if mse < min_mse:
        min_mse = mse

In [ ]:
min_mse